import Pkg
Pkg.add("CSV")
Pkg.add("StatsBase")
Pkg.add("DataFrames")
Pkg.add("UMAP")
Pkg.add("RCall")

## Settings

Load packages

In [ ]:
using CSV, StatsBase, Statistics, DataFrames, UMAP, RCall, Random
using Dates: now

In [ ]:
@rlibrary ggplot2
@rlibrary extrafont

In [ ]:
R"""
# Customize ggplot appearance

library(ggplot2)
library(extrafont)

# Load extra fonts
loadfonts()

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
scale_fill_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)
"""

## Loading

Load generated datasets and combine them

In [ ]:
R = CSV.read("Data/matR.csv", header = false) # Reference
N = CSV.read("Data/matN.csv", header = false) # Negative control
PS = CSV.read("Data/matPS.csv", header = false) # Shifted
PR = CSV.read("Data/matPR.csv", header = false) # Rescaled
1 # Do not display last element

In [ ]:
dataset = vcat(R, N, PS, PR)

In [ ]:
# Remember how these records were generated
origDataset = vcat(repeat(["Reference"], size(R, 1)),
                   repeat(["Negative control"], size(N, 1)),
                   repeat(["Shifted control"], size(PS, 1)),
                   repeat(["Rescaled control"], size(PR, 1)))

## Feature filtering

In [ ]:
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables
dimUMAP = 5

In [ ]:
# Center and scale on control values
transfNorm(x,y) = (x .- median(y)) ./ mad(y)
indRef = origDataset .== "Reference"
normDataset = DataFrame(map(x -> transfNorm(x, x[indRef]), eachcol(dataset)))
1

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(reference) = 1, it means that we rank features by how more variable they are
# for all conditions compared to the reference
orderFt = sortperm(convert(Array, map(x -> mad(x, normalize = true), eachcol(normDataset))), rev=true)

In [ ]:
function decorrelate(data::DataFrame; orderCol = nothing, threshold = 0.8)
    """Returns column  of 'data' that are never pairwise-correlated more than 'threshold',
    prioritizing columns by a giver order 'orderCol' (defaults to left to right).
    """
    if isnothing(orderCol)
        orderCol = 1:size(normDataset, 2)
    end
    # Columns to sort
    L1 = orderCol
    # Sorted columns to keep
    L2 = Array{Int64,1}()
    while length(L1) > 0
        refFt = first(L1)
        append!(L2, refFt)
        popfirst!(L1)
        stillToKeep = []
        for (ift, ft) in enumerate(L1)
            if abs(cor(data[refFt], data[ft])) < threshold
                append!(stillToKeep, ift)
            end
        end
        L1 = L1[stillToKeep]
    end
    return(L2)
end

In [ ]:
uncorrFt = decorrelate(normDataset, orderCol = orderFt, threshold = FILT_MAX_CORR)
normDataset = normDataset[uncorrFt]

In [ ]:
# Visualization
Random.seed!(1234)
global umND = umap(convert(Matrix, normDataset)', 2; min_dist = 0.01, n_epochs = 200)
umND = convert(DataFrame, umND')
names!(umND, [:UMAP1, :UMAP2])

umND[:Condition] = origDataset

gp = ggplot(umND, aes(:UMAP1, :UMAP2)) + geom_point(aes(color = :Condition), alpha = 0.3)
ggsave("Fig/UMAP_test.pdf", gp)
1